In [ ]:
# This dataset analyzes the COVID-19 data for Colombia and builds the GeoJSON map counting active cases by department
# Author: Fabian Peña
# Creation date:
# Last modification date: 19/07/2020

In [ ]:
import datetime

import pandas as pd
#import geopandas as gpd
from google.cloud import storage


from sodapy import Socrata

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from pandas_profiling import ProfileReport

# INS data

In [ ]:
# Creating the Socrata client
client = Socrata( 'www.datos.gov.co', None )

In [ ]:
# Loading the COVID-19 data for Colombia
results = client.get( 'gt2j-8ykr', limit = 1000000 )
covid = pd.DataFrame.from_records(results)
covid.rename( columns = { 'id_de_caso': 'ID caso',
                         'fecha_de_notificaci_n': 'Fecha notificacion',
                         'c_digo_divipola': 'Codigo ciudad',
                         'ciudad_de_ubicaci_n': 'Ciudad',
                         'departamento': 'Departamento',
                         'atenci_n': 'Atencion',
                         'edad': 'Edad',
                         'sexo': 'Sexo',
                         'tipo': 'Tipo',
                         'estado': 'Estado',
                         'pa_s_de_procedencia': 'Pais procedencia',
                         'fis': 'Fecha inicio sintomas',
                         'fecha_de_muerte': 'Fecha muerte',
                         'fecha_diagnostico': 'Fecha diagnostico',
                         'fecha_recuperado': 'Fecha recuperacion',
                         'fecha_reporte_web': 'Fecha reporte web',
                         'tipo_recuperaci_n': 'Tipo recuperacion' }, inplace = True )

In [ ]:
# Getting the size of the df
covid.shape

In [ ]:
# Add a new column for asimptomatic observations
covid.loc[ covid[ 'Fecha inicio sintomas' ] == 'Asintomático', 'Asintomatico' ] = True
covid.loc[ covid[ 'Fecha inicio sintomas' ] != 'Asintomático', 'Asintomatico' ] = False
covid[ 'Asintomatico' ] = covid[ 'Asintomatico' ].astype( bool )

In [ ]:
# Casting date columns
covid[ 'Fecha notificacion' ] = pd.to_datetime( pd.to_datetime( covid[ 'Fecha notificacion' ] ).dt.date )
covid[ 'Fecha inicio sintomas' ] = pd.to_datetime( pd.to_datetime( covid[ 'Fecha inicio sintomas' ], errors = 'coerce' ).dt.date )
covid[ 'Fecha muerte' ] = pd.to_datetime( pd.to_datetime( covid[ 'Fecha muerte' ], errors = 'coerce' ).dt.date )
covid[ 'Fecha diagnostico' ] = pd.to_datetime( pd.to_datetime( covid[ 'Fecha diagnostico' ], errors = 'coerce' ).dt.date )
covid[ 'Fecha recuperacion' ] = pd.to_datetime( pd.to_datetime( covid[ 'Fecha recuperacion' ], errors = 'coerce' ).dt.date )
covid[ 'Fecha reporte web' ] = pd.to_datetime( pd.to_datetime( covid[ 'Fecha reporte web' ] ).dt.date )

In [ ]:
# Unifying values for sex
covid[ 'Sexo' ].replace( { 'm': 'M', 'f': 'F' }, inplace = True )

In [ ]:
# Unifying values for status
covid[ 'Estado' ].replace( { 'leve': 'Leve', 'LEVE': 'Leve' }, inplace = True )

In [ ]:
# Unifying values for atention
covid[ 'Atencion' ].replace( { 'casa': 'Casa', 'N/A': None }, inplace = True )

In [ ]:
# Unifying values for procedence
covid[ 'Pais procedencia' ].replace( { '-   -': None }, inplace = True )

In [ ]:
covid.loc[ covid[ 'Fecha recuperacion' ].notnull(), 'Recuperado' ] = True
covid.loc[ covid[ 'Fecha recuperacion' ].isnull(), 'Recuperado' ] = False

In [ ]:
covid.loc[ covid[ 'Fecha muerte' ].notnull(), 'Fallecido' ] = True
covid.loc[ covid[ 'Fecha muerte' ].isnull(), 'Fallecido' ] = False

In [ ]:
covid[ 'Edad' ] = covid[ 'Edad' ].astype( int )

In [ ]:
def get_age_group( x ):
    if x <= 19:
        return '0 - 19'
    elif x > 19 and x <= 39:
        return '20 - 39'
    elif x > 39 and x <= 59:
        return '40 - 59'
    elif x > 59 and x <= 79:
        return '60 - 79'
    elif x > 79:
          return '80+'
    else:
        return None

covid[ 'Grupo edad' ] = covid[ 'Edad' ].apply( get_age_group )

In [ ]:
# Getting the column types
covid.dtypes

In [ ]:
# Showing the firts rows
covid.tail()

In [ ]:
covid[ 'Fecha diagnostico' ].hist()

In [ ]:
# How many observations have null atention
covid.loc[ covid[ 'Atencion' ].isnull() ].shape

In [ ]:
# How many observations have null status
covid.loc[ covid[ 'Estado' ].isnull() ].shape

In [ ]:
del covid[ 'nombre_grupo_etnico' ]
del covid[ 'pertenencia_etnica' ]
del covid[ 'codigo_pais' ]

In [ ]:
covid_copy = covid.copy()

In [ ]:
del covid_copy[ 'codigo_departamento' ]

In [ ]:
#covid_copy.to_csv( './data/covid19.csv', index = False )

In [ ]:
#DataFrame to CSV
csv_data=covid_copy.to_csv(index=False)

#Client, bucket and blob to acces gcloud
client= storage.Client()
bucket= client.get_bucket('siscovid')
blob=bucket.blob('covid19.csv')

#Uploads the file to gcloud
blob.upload_from_string(data=csv_data, content_type='text/csv')


# Indicators

In [ ]:
covid[ 'ID caso' ].nunique()

In [ ]:
covid.loc[ covid[ 'Recuperado' ] == True ].shape[ 0 ]

In [ ]:
covid.loc[ covid[ 'Fallecido' ] == True ].shape[ 0 ]

In [ ]:
covid.loc[ ( covid[ 'Atencion' ] == 'Hospital' ) ].shape[ 0 ]

In [ ]:
covid.loc[ ( covid[ 'Atencion' ] == 'Hospital UCI' ) ].shape[ 0 ]

# Time series

In [ ]:
# Plotting diagnosed cases by day
covid.groupby( [ covid[ 'Fecha inicio sintomas' ].dt.date ] ).agg( { 'ID caso': 'count' } ).plot( figsize = ( 20, 8 ) )

In [ ]:
CITIES = [ 'Bogotá D.C.', 'Barranquilla', 'Cali', 'Cartagena de Indias', 'Medellín' ]

In [ ]:
daily_cases_by_city = covid.loc[ covid[ 'Ciudad' ].isin( CITIES ) ].groupby( [ 'Ciudad', covid[ 'Fecha inicio sintomas' ].dt.date ] ).agg( { 'ID caso': 'count' } ).unstack( level = 0 ).fillna( 0 )
daily_cases_by_city.columns = daily_cases_by_city.columns.droplevel()
daily_cases_by_city.reset_index( inplace = True )

In [ ]:
daily_cases_by_city.dtypes

In [ ]:
plt.figure( figsize = ( 20, 8 ) )
sns.lineplot( x = 'Fecha inicio sintomas', y = 'ID caso', hue = 'Ciudad', data = covid.loc[ covid[ 'Ciudad' ].isin( CITIES ) ].groupby( [ 'Ciudad', covid[ 'Fecha inicio sintomas' ].dt.date ] ).agg( { 'ID caso': 'count' } ).reset_index() )

In [ ]:
daily_cases_by_city.rename( columns = { 'Fecha inicio sintomas': 'date', 'Cartagena de Indias': 'Cartagena' } ).to_csv( './daily-cases-viz/data/daily-cases-by-city.csv', index = False )

In [ ]:
deaths_by_city = covid.loc[ covid[ 'Ciudad' ].isin( CITIES ) ].groupby( [ 'Ciudad', covid[ 'Fecha muerte' ].dt.date ] ).agg( { 'ID caso': 'count' } ).unstack( level = 0 ).fillna( 0 )
deaths_by_city.columns = deaths_by_city.columns.droplevel()
deaths_by_city.reset_index( inplace = True )

In [ ]:
deaths_by_city.tail()

In [ ]:
deaths_by_city.dtypes

In [ ]:
plt.figure( figsize = ( 20, 8 ) )
sns.lineplot( x = 'Fecha muerte', y = 'ID caso', hue = 'Ciudad', data = covid.loc[ covid[ 'Ciudad' ].isin( CITIES ) ].groupby( [ 'Ciudad', covid[ 'Fecha muerte' ].dt.date ] ).agg( { 'ID caso': 'count' } ).reset_index() )

In [ ]:
deaths_by_city.rename( columns = { 'Fecha muerte': 'date', 'Cartagena de Indias': 'cartagena', 'Barranquilla': 'barranquilla', 'Bogotá D.C.': 'bogota', 'Cali': 'cali', 'Medellín': 'medellin' } ).to_csv( './data/meta/deaths-by-city.csv', index = False )

In [ ]:
# Plotting diagnosed cases by day (cumulative sum)
covid.groupby( [ covid[ 'Fecha diagnostico' ].dt.date ] ).agg( { 'ID caso': 'count' } ).cumsum().plot( figsize = ( 20, 8 ) )

In [ ]:
# Preparing and merging the groupings in one dataframe

diagnosed = covid.groupby( [ covid[ 'Fecha diagnostico' ].dt.date ] ) \
    .agg( { 'ID caso': 'count' } ) \
    .cumsum() \
    .rename( columns = { 'Fecha diagnostico': 'Fecha', 'ID caso': 'Diagnosticados' } )

recovered = covid.groupby( [ covid[ 'Fecha recuperacion' ].dt.date ] )  \
    .agg( { 'ID caso': 'count' } ) \
    .cumsum() \
    .rename( columns = { 'Fecha recuperacion': 'Fecha', 'ID caso': 'Recuperados' } )

deaths = covid.groupby( [ covid[ 'Fecha muerte' ].dt.date ] )  \
    .agg( { 'ID caso': 'count' } ) \
    .cumsum() \
    .rename( columns = { 'Fecha muerte': 'Fecha', 'ID caso': 'Fallecidos' } )

infections = diagnosed.merge( recovered, how = 'outer', left_index = True, right_index = True )
infections = infections.merge( deaths, how = 'outer', left_index = True, right_index = True )
infections.fillna( 0, inplace = True )

In [ ]:
infections.tail()

In [ ]:
# Plotting diagnosed and recovered cases by day
infections.plot( figsize = ( 20, 8 ) )

In [ ]:
# Calculating the difference between diagnosed and recovered cases
infections[ 'Casos activos' ] = infections[ 'Diagnosticados' ] - infections[ 'Recuperados' ]

In [ ]:
infections.tail()

In [ ]:
# Plotting difference cases by day
infections[ 'Casos activos' ].plot( figsize = ( 20, 8 ) )

# Geo

In [ ]:
covid.head()

In [ ]:
covid[ 'Atencion' ].value_counts( dropna = False )

In [ ]:
covid[ 'codigo_departamento' ] = covid[ 'codigo_departamento' ].astype( int )

In [ ]:
cases_by_depto = covid.loc[ ~covid[ 'Atencion' ].isin( [ 'Recuperado', 'Fallecido' ] ) ].groupby( 'codigo_departamento' ).agg( { 'ID caso': 'count' } ).reset_index().rename( columns = { 'ID caso': 'Casos' } )

In [ ]:
cases_by_depto.dtypes

In [ ]:
cases_by_depto.head()

In [ ]:
departamentos = gpd.read_file( './data/geo/colombia/departamentos.json' )

In [ ]:
departamentos.head()

In [ ]:
cases_by_depto = departamentos.merge( cases_by_depto, how = 'left', on = 'codigo_departamento' )
cases_by_depto[ 'Casos' ] = cases_by_depto[ 'Casos' ].fillna( 0 )

In [ ]:
cases_by_depto

In [ ]:
# Removing San Andrés
cases_by_depto = cases_by_depto.drop( [ 0 ] )

In [ ]:
cases_by_depto.to_file( './colombia-map/data/departamentos.json', driver ='GeoJSON' )

In [ ]:
covid.head()

In [ ]:
deaths_bogota = covid.loc[ covid[ 'Ciudad' ] == 'Bogotá D.C.' ].groupby( [ covid[ 'Fecha muerte' ].dt.date ] )  \
    .agg( { 'ID caso': 'count' } ) \
    .cumsum().reset_index() \
    .rename( columns = { 'Fecha muerte': 'date', 'ID caso': 'Fallecidos_reales' } )

In [ ]:
deaths_bogota.to_csv( './meta-viz/data/bogota-deaths.csv', index = False )

In [ ]:
deaths_cali = covid.loc[ covid[ 'Ciudad' ] == 'Cali' ].groupby( [ covid[ 'Fecha muerte' ].dt.date ] )  \
    .agg( { 'ID caso': 'count' } ) \
    .cumsum().reset_index() \
    .rename( columns = { 'Fecha muerte': 'date', 'ID caso': 'Fallecidos_reales' } )

In [ ]:
deaths_cali.to_csv( './meta-viz/data/cali-deaths.csv', index = False )

In [ ]:
deaths_barranquilla = covid.loc[ covid[ 'Ciudad' ] == 'Barranquilla' ].groupby( [ covid[ 'Fecha muerte' ].dt.date ] )  \
    .agg( { 'ID caso': 'count' } ) \
    .cumsum().reset_index() \
    .rename( columns = { 'Fecha muerte': 'date', 'ID caso': 'Fallecidos_reales' } )

In [ ]:
deaths_barranquilla.to_csv( './meta-viz/data/barranquilla-deaths.csv', index = False )

In [ ]:
covid.rename( columns = { 'Fecha diagnostico': 'Fecha de diagnóstico', 'Ciudad de ubicación': 'Ciudad de residencia', 'Tipo': 'Tipo de caso', 'atención': 'Ubicación' }, inplace = True )

In [ ]:
def get_age_group( x ):
    if x <= 19:
        return '0 - 19'
    elif x > 19 and x <= 39:
        return '20 - 39'
    elif x > 39 and x <= 59:
        return '40 - 59'
    elif x > 59 and x <= 79:
        return '60 - 79'
    elif x > 79:
          return '80+'
    else:
        return None

covid[ 'Grupo de edad' ] = covid[ 'Edad' ].apply( get_age_group )

In [ ]:
covid[ 'Localidad de residencia' ] = None

In [ ]:
columns = [ 'ID de caso', 'Fecha de diagnóstico', 'Ciudad de residencia', 'Localidad de residencia', 'Edad', 'Sexo', 'Tipo de caso', 'Ubicación', 'Estado', 'Grupo de edad' ]

In [ ]:
covid = covid[ columns ]

# Barranquilla

In [ ]:
barranquilla = covid.loc[ covid[ 'Ciudad de residencia' ] == 'Barranquilla' ]

In [ ]:
barranquilla.shape

In [ ]:
barranquilla.dtypes

In [ ]:
barranquilla.tail()

In [ ]:
barranquilla[ 'Ubicación' ].value_counts( dropna = False )

In [ ]:
barranquilla.loc[ ~barranquilla[ 'Ubicación' ].isin( [ 'Recuperado', 'Fallecido' ] ) ].shape

In [ ]:
barranquilla.loc[ barranquilla[ 'Ubicación' ] == 'Hospital UCI' ]

In [ ]:
barranquilla.to_csv( './data/covid19_barranquilla.csv', index = False )

# Medellín

In [ ]:
medellin = covid.loc[ covid[ 'Ciudad de residencia' ] == 'Medellín' ]

In [ ]:
medellin.shape

In [ ]:
medellin.dtypes

In [ ]:
medellin.tail()

In [ ]:
medellin[ 'Ubicación' ].value_counts( dropna = False )

In [ ]:
medellin.to_csv( './data/covid19_medellin.csv', index = False )

# Cali

In [ ]:
cali = covid.loc[ covid[ 'Ciudad de residencia' ] == 'Cali' ]

In [ ]:
cali.shape

In [ ]:
cali.dtypes

In [ ]:
cali.tail()

In [ ]:
cali[ 'Ubicación' ].value_counts( dropna = False )

In [ ]:
cali.to_csv( './data/covid19_cali.csv', index = False )

# Cartagena

In [ ]:
cartagena = covid.loc[ covid[ 'Ciudad de residencia' ] == 'Cartagena de Indias' ]

In [ ]:
cartagena.shape

In [ ]:
cartagena.dtypes

In [ ]:
cartagena.tail()

In [ ]:
cartagena[ 'Ubicación' ].value_counts( dropna = False )

In [ ]:
cartagena.to_csv( './data/covid19_cartagena.csv', index = False )

In [ ]:
covid[ 'dias_para_diagnostico' ] = ( covid[ 'Fecha_diagnostico' ] - covid[ 'FIS' ] ).dt.days

In [ ]:
covid[ 'Estado' ].value_counts( dropna = False )

In [ ]:
plt.figure( figsize = ( 20, 8 ) )
ax = sns.boxplot( data = covid, x = 'dias_para_diagnostico', y = 'Estado' )

In [ ]:
covid[ 'atención' ].value_counts( dropna = False )

In [ ]:
covid.loc[ covid[ 'atención' ] == 'Casa' ][ 'Edad' ].quantile( [ 0.25, 0.5, 0.75 ] )

In [ ]:
plt.figure( figsize = ( 20, 8 ) )
ax = sns.boxplot( data = covid, x = 'Edad', y = 'atención' )

In [ ]:
covid.loc[ covid[ 'Tipo' ] == 'Importado' ].groupby( 'Fecha_diagnostico' ).agg( { 'ID_de_caso': 'count' } ).plot( kind = 'bar', figsize = ( 20, 8 ) )

In [ ]:
# Preparing and merging the groupings in one dataframe

diagnosed = covid.loc[ covid[ 'Tipo' ] == 'Importado' ] \
    .groupby( 'Fecha_diagnostico' ) \
    .agg( { 'ID_de_caso': 'count' } ) \
    .rename( columns = { 'Fecha_diagnostico': 'Fecha', 'ID_de_caso': 'Diagnosticados' } )

recovered = covid.loc[ covid[ 'Tipo' ] == 'Importado' ] \
    .groupby( 'FIS' ) \
    .agg( { 'ID_de_caso': 'count' } ) \
    .rename( columns = { 'Fecha_diagnostico': 'Fecha', 'ID_de_caso': 'Inicio de sintomas' } )

infections = diagnosed.merge( recovered, how = 'outer', left_index = True, right_index = True )
infections.fillna( 0, inplace = True )

In [ ]:
infections.tail()

In [ ]:
f, axarr = plt.subplots( 2, sharex = True, figsize = ( 20, 10 ) )
axarr[ 0 ].set_title( 'Fecha de diagnostico de casos importados' )
infections[ 'Diagnosticados' ].plot( kind = 'bar', ax = axarr[ 0 ] )
axarr[ 1 ].set_title( 'Fecha de inicio de sintomas de casos importados' )
infections[ 'Inicio de sintomas' ].plot( kind = 'bar', ax = axarr[ 1 ] )
axarr[ 0 ].axvline( x = 23.5, color = 'r', linestyle = '--' )
axarr[ 1 ].axvline( x = 23.5, color = 'r', linestyle = '--' )

In [ ]:
covid[ 'Estado' ].value_counts( dropna = False )

In [ ]:
covid[ 'dias_para_recuperarse' ] = ( covid[ 'Fecha_recuperado' ] - covid[ 'Fecha_diagnostico' ] ).dt.days

In [ ]:
plt.figure( figsize = ( 20, 8 ) )
ax = sns.boxplot( data = covid, x = 'dias_para_recuperarse', y = 'Estado' )

In [ ]:
covid.loc[ covid[ 'dias_para_recuperarse' ] < 0 ]

In [ ]:
covid.head()

In [ ]:
covid.to_csv( './data/covid_clean.csv', index = False, sep = ',' )

In [ ]:
infections.reset_index().to_csv( './data/covid_groups.csv', index = False, sep = ',' )

In [ ]:
departamentos.to_file( './data/departamentos_groups.json', driver = 'GeoJSON' )